<a href="https://colab.research.google.com/github/elleniayele/salespredicationspca/blob/main/pridictionofsaleswithpca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
set_config(display='diagram')

In [ ]:
import pandas as pd
filename ='/content/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
df['Outlet_Identifier'].unique()

array(['OUT049', 'OUT018', 'OUT010', 'OUT013', 'OUT027', 'OUT045',
       'OUT017', 'OUT046', 'OUT035', 'OUT019'], dtype=object)

Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)

In [ ]:
df.shape

(8523, 12)

In [ ]:
df['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [ ]:
for col in df:
 print(col)
 print(df[col].unique())
 print('\n\n')

Item_Identifier
['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']



Item_Weight
[ 9.3    5.92  17.5   19.2    8.93  10.395 13.65     nan 16.2   11.8
 18.5   15.1   17.6   16.35   9.    13.35  18.85  14.6   13.85  13.
  7.645 11.65   5.925 19.25  18.6   18.7   17.85  10.     8.85   9.8
 13.6   21.35  12.15   6.42  19.6   15.85   7.39  10.195  9.895 10.895
  7.905  9.195  8.365  7.97  17.7   19.35   8.645 15.6   18.25   7.855
  7.825  8.39  12.85  19.     5.905  7.76  16.75  15.5    6.055  6.305
 20.85  20.75   8.895 19.7    8.75  13.3    8.31  19.75  17.1   10.5
  6.635 14.15   8.89   9.1    7.5   16.85   7.485 11.6   12.65  20.25
  8.6   12.6    8.88  20.5   13.5    7.235  6.92   8.02  12.8   16.6
 14.    16.    21.25   7.365 18.35   5.465  7.27   6.155 19.5   15.2
 14.5   13.1   12.3   11.1   11.3    5.75  11.35   6.525 10.3    5.78
 11.85  18.75   5.26  16.1    9.5   13.8   14.65   6.67   6.11  17.2
  6.32   4.88   5.425 14.1    7.55  17.25  12.    10.1    7.785 13.15
  8.5    7

we have to fix inconsistencies with item_fat_content column

In [ ]:
df['Item_Fat_Content']=df['Item_Fat_Content'].str.lower()

In [ ]:
df=df.replace(['lf','reg'],['low fat','regular'])
df['Item_Fat_Content'].unique()

array(['low fat', 'regular'], dtype=object)

now lets convert the ordinal type to ordered numeric type

In [ ]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [ ]:
df.drop(columns='Item_Identifier', inplace= True)

In [ ]:
df.drop(columns=['Item_Visibility','Outlet_Establishment_Year'], inplace= True, axis=1)

In [ ]:
X = df.drop(columns = ['Item_Outlet_Sales'])
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [ ]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
reg = LinearRegression()

In [ ]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [ ]:
#catagorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [ ]:
# Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple,remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])

In [ ]:
#Function for metrics
def eval_regression(y_true, y_pred, name='model'):
  """Takes true targets and predictions from a regression model and prints
  MAE, MSE, RMSE, AND R2 scores
  Set 'name' to name of model and 'train' or 'test' as appropriate"""
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_true, y_pred)

  print(f'{name} Scores')
  print(f'MAE: {mae:,.4f} \nMSE: {mse:,.4f} \nRMSE: {rmse:,.4f} \nR2: {r2:.4f}\n')#

In [ ]:
preprocessor.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])

In [ ]:
#transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
X_train_processed

array([[ 0.81724868,  1.82810922,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5563395 ,  0.60336888,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.13151196,  0.24454056,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.11373638,  1.52302674,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.76600931, -0.38377708,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81724868, -0.73836105,  1.        , ...,  1.        ,
         0.        ,  0.        ]])

build a linear regression model

In [ ]:
linreg_pipe = make_pipeline(preprocessor,LinearRegression())
linreg_pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                ('linearregression', LinearRegression())])

 Make predictions using the testing data.

In [ ]:
training_predictions = linreg_pipe.predict(X_train)
test_predictions = linreg_pipe.predict(X_test)
eval_regression(y_train,training_predictions)
eval_regression(y_test,test_predictions)

model Scores
MAE: 847.2653 
MSE: 1,297,982.4269 
RMSE: 1,139.2903 
R2: 0.5614

model Scores
MAE: 803.3697 
MSE: 1,193,776.3285 
RMSE: 1,092.6007 
R2: 0.5673



Regression tree model to predict sales.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dec_tree = DecisionTreeRegressor(random_state = 42)

In [ ]:
Dec_tree_reg_pipe = make_pipeline(preprocessor,dec_tree)
Dec_tree_reg_pipe .fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [ ]:
# Make predictions using the training and testing data
training_predictions2 = Dec_tree_reg_pipe .predict(X_train)
test_predictions2 = Dec_tree_reg_pipe .predict(X_test)
training_predictions2[:10]

array([ 515.3292, 3056.022 , 1577.946 , 1331.6   , 1687.1372,  111.8544,
       1151.1682, 3401.5722, 3570.0196, 1523.3504])

Compare the performance of your model based on different metrics

In [ ]:
eval_regression(y_train,training_predictions2)
eval_regression(y_test,test_predictions2)

model Scores
MAE: 0.2316 
MSE: 171.3562 
RMSE: 13.0903 
R2: 0.9999

model Scores
MAE: 1,052.2769 
MSE: 2,373,413.2107 
RMSE: 1,540.5886 
R2: 0.1397



lets tune the model

In [ ]:
Dec_tree_reg_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sp

In [ ]:
params = { 'decisiontreeregressor__random_state': [0,10,15,20,25,30,35,42]}

In [ ]:
Dec_treegs = GridSearchCV(Dec_tree_reg_pipe, params)
Dec_treegs.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                       ('decisiontreeregressor',
                                        DecisionTreeRegressor(random_state=42))]),
             param_grid={'decisiontreeregressor__random_state': [0, 10, 15, 20,
                                                                 25, 30, 35,
                                                                 42]})

In [ ]:
Dec_treegs.best_params_

{'decisiontreeregressor__random_state': 35}

In [ ]:
best_Dec = Dec_treegs.best_estimator_

In [ ]:
train_preds22 = best_Dec.predict(X_train)
test_preds22 = best_Dec.predict(X_test)

In [ ]:
eval_regression(y_train,train_preds22)
eval_regression(y_test,test_preds22)

model Scores
MAE: 0.2316 
MSE: 171.3562 
RMSE: 13.0903 
R2: 0.9999

model Scores
MAE: 1,035.0055 
MSE: 2,290,603.3872 
RMSE: 1,513.4739 
R2: 0.1698



using bagged tree regressor model

In [ ]:
from sklearn.ensemble import BaggingRegressor
bagreg = BaggingRegressor(random_state = 42)

In [ ]:
bag_reg_pipe = make_pipeline(preprocessor,bagreg)
bag_reg_pipe .fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                ('baggingregressor', BaggingRegressor(random_state=42))])

In [ ]:
# Make predictions using the training and testing data
training_predictions3 = bag_reg_pipe .predict(X_train)
test_predictions3 = bag_reg_pipe .predict(X_test)
training_predictions2[:10]

array([ 515.3292, 3056.022 , 1577.946 , 1331.6   , 1687.1372,  111.8544,
       1151.1682, 3401.5722, 3570.0196, 1523.3504])

In [ ]:
eval_regression(y_train,training_predictions3)
eval_regression(y_test,test_predictions3)

model Scores
MAE: 322.2588 
MSE: 245,158.8581 
RMSE: 495.1352 
R2: 0.9172

model Scores
MAE: 810.4387 
MSE: 1,379,493.7155 
RMSE: 1,174.5185 
R2: 0.5000



use hyperparameters to tune the model

In [ ]:
bag_reg_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sp

In [ ]:
params = {'baggingregressor__random_state': [10,15,20,25,30,35,40,42],
          'baggingregressor__n_estimators': [1,2,3,5,7,8,10]}

In [ ]:
bag_reg22 = GridSearchCV(bag_reg_pipe, params)
bag_reg22.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          Sim...t_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                       ('baggingregressor',
                                        BaggingRegressor(random_state=42))]),
             param_grid={'baggingregressor__n_estimators': [1, 2, 3, 5, 7, 8,
                                                            10],
                         'baggingregressor__random_state': [10, 15, 20, 25, 30,
                                                            35, 40, 42]})

In [ ]:
bag_reg22.best_params_

{'baggingregressor__n_estimators': 10, 'baggingregressor__random_state': 10}

we have best values at baggingregressor__n_estimators': 10, and 'baggingregressor__random_state': 10

In [ ]:
bestbagreg = bag_reg22.best_estimator_

In [ ]:
train_predsbg = bestbagreg .predict(X_train)
test_predsbg= bestbagreg .predict(X_test)

In [ ]:
eval_regression(y_train,train_predsbg)
eval_regression(y_test,test_predsbg)

model Scores
MAE: 326.1809 
MSE: 259,547.4269 
RMSE: 509.4580 
R2: 0.9123

model Scores
MAE: 804.4247 
MSE: 1,322,351.5511 
RMSE: 1,149.9355 
R2: 0.5207



using randomforest regressor



In [ ]:
rf = RandomForestRegressor(random_state = 42)

In [ ]:
rf_pipe = make_pipeline(preprocessor,rf)
rf_pipe .fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [ ]:
training_predictions4 = rf_pipe .predict(X_train)
test_predictions4 = rf_pipe .predict(X_test)
training_predictions4[:10]

array([1881.291138, 3013.304272, 2074.546246, 1406.782136, 2008.212592,
        162.72152 , 1317.078902, 3589.92702 , 3613.36318 , 1580.236352])

In [ ]:
eval_regression(y_train,training_predictions4)
eval_regression(y_test,test_predictions4)

model Scores
MAE: 299.6640 
MSE: 188,834.4863 
RMSE: 434.5509 
R2: 0.9362

model Scores
MAE: 787.1621 
MSE: 1,287,482.7648 
RMSE: 1,134.6730 
R2: 0.5333



lets tune the model

In [ ]:
rf_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sp

In [ ]:
params = {'randomforestregressor__n_estimators':[0,10,20,30,40,50,55,60.65,70,75,80,90,100]}

In [ ]:
rf_gs = GridSearchCV(rf_pipe, params)
rf_gs.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          Sim...ter(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             param_grid={'randomforestregressor__n_estimators': [0, 10, 20, 30,
                                                                 40, 50, 55,
                                                                 60.65, 70, 75,
                                                                 80, 90, 100]})

In [ ]:
rf_gs .best_params_

{'randomforestregressor__n_estimators': 90}

In [ ]:
best_rf = rf_gs.best_estimator_


In [ ]:
train_predsg= best_rf .predict(X_train)
test_predsg =best_rf .predict(X_test)

In [ ]:
eval_regression(y_train,train_predsg)

model Scores
MAE: 299.9588 
MSE: 189,449.9120 
RMSE: 435.2584 
R2: 0.9360



In [ ]:
eval_regression(y_test,test_predsg)

model Scores
MAE: 787.4692 
MSE: 1,288,818.7584 
RMSE: 1,135.2615 
R2: 0.5329



Try feature engineering techniques or PCA to engineer your dataset to see how this impacts your modeling results.
We encourage you to try multiple techniques depending on your data!

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=.96)
rf_pipe1 = make_pipeline(preprocessor,pca)

In [ ]:
#rf_pipe2= make_pipeline(rf_pipe1, best_rf
rf_pipe1.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                ('pca', PCA(n_components=0.96))])

using linear regression for pca

In [ ]:
linreg_pipe = make_pipeline(rf_pipe1,LinearRegression())
linreg_pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('pipeline-1',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer()),
                                                                                   ('standardscaler',
                                                                                    StandardScaler())]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                  ('pipeline-2',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('onehotencoder',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                 ('pca', PCA(n_components=0.96))])),
                ('linearregression', LinearRegression())])

In [ ]:
training_predictionspca1= linreg_pipe.predict(X_train)
test_predictionspca1 = linreg_pipe.predict(X_test)
eval_regression(y_train,training_predictionspca1)
eval_regression(y_test,test_predictionspca1)

model Scores
MAE: 847.6287 
MSE: 1,298,860.2563 
RMSE: 1,139.6755 
R2: 0.5611

model Scores
MAE: 802.4341 
MSE: 1,191,375.1000 
RMSE: 1,091.5013 
R2: 0.5682



using decision tree model with pca

In [ ]:
dec_tree= make_pipeline(rf_pipe1,dec_tree)
dec_tree.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('pipeline-1',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer()),
                                                                                   ('standardscaler',
                                                                                    StandardScaler())]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                  ('pipeline-2',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('onehotencoder',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                 ('pca', PCA(n_components=0.96))])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [ ]:
training_predictionspca2= dec_tree.predict(X_train)
test_predictionspca2 = dec_tree.predict(X_test)
eval_regression(y_train,training_predictionspca2)
eval_regression(y_test,test_predictionspca2)

model Scores
MAE: 0.2316 
MSE: 171.3562 
RMSE: 13.0903 
R2: 0.9999

model Scores
MAE: 1,018.4722 
MSE: 2,191,836.9955 
RMSE: 1,480.4854 
R2: 0.2056



bagging regressor with pca

In [ ]:
bag_regpca= make_pipeline(rf_pipe1,bagreg)
bag_regpca.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('pipeline-1',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer()),
                                                                                   ('standardscaler',
                                                                                    StandardScaler())]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                  ('pipeline-2',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('onehotencoder',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                 ('pca', PCA(n_components=0.96))])),
                ('baggingregressor', BaggingRegressor(random_state=42))])

In [ ]:
training_predictionspca3= bag_regpca.predict(X_train)
test_predictionspca3 = bag_regpca.predict(X_test)
eval_regression(y_train,training_predictionspca3)
eval_regression(y_test,test_predictionspca3)

model Scores
MAE: 326.1596 
MSE: 253,019.3226 
RMSE: 503.0103 
R2: 0.9145

model Scores
MAE: 792.9185 
MSE: 1,313,441.6560 
RMSE: 1,146.0548 
R2: 0.5239



using using randomforest regressor pca

In [ ]:
randforst= make_pipeline(rf_pipe1,rf)
randforst.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('pipeline-1',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer()),
                                                                                   ('standardscaler',
                                                                                    StandardScaler())]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be43d0>),
                                                                  ('pipeline-2',
                                                                   Pipeline(steps=[('simpleimputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('onehotencoder',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f1c72be4ee0>)])),
                                 ('pca', PCA(n_components=0.96))])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [ ]:
training_predictionspca4= randforst.predict(X_train)
test_predictionspca4= randforst.predict(X_test)
eval_regression(y_train,training_predictionspca4)
eval_regression(y_test,test_predictionspca4)

model Scores
MAE: 303.0307 
MSE: 194,495.5551 
RMSE: 441.0165 
R2: 0.9343

model Scores
MAE: 771.2808 
MSE: 1,263,310.4449 
RMSE: 1,123.9708 
R2: 0.5421



the random forest model has better results than the other models